This notebook generates the data files containing PDF(z), which are required by Prospector-$\beta$.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

In [ ]:
import os, sys
import numpy as np
from scipy.interpolate import interp1d, UnivariateSpline
from scipy.ndimage import gaussian_filter1d
from astropy.table import Table
from astropy.cosmology import WMAP9, z_at_value
import astropy.units as u

sys.path.append('../prospect/models/')
import priors as PB

In [ ]:
const_phi = True # Leja+20 mass functions
# const_phi = False # Leja+20 at z < 3 & transiting to Tacchella+18 mass functions

In [ ]:
# read in the mass complete limits that are estimated from the mock catalog
# see Sec. 3.2 of Wang, Leja, et al. 2023
# in principle these should be re-generated for different data sets
prior_data_dir = '../prospect/models/prior_data/'
z_bins_ctr, mc_ctr = np.loadtxt(prior_data_dir+'mc_from_mocks.txt', dtype=float, unpack=True)

def mass_completion_at_z_from_cat(zred):
    if zred <= 10 and zred >= 1:
        return np.poly1d(np.polyfit(z_bins_ctr, mc_ctr, 3))(zred)
    elif zred < 1:
        return np.poly1d(np.polyfit(z_bins_ctr[:3], mc_ctr[:3], 1))(zred)
    else:
        fintp_z_mc = interp1d(z_bins_ctr, mc_ctr, kind='nearest', fill_value="extrapolate")
        return fintp_z_mc(zred)

zgrid = np.linspace(0., 20, 101)

from_fintp_z_mc = []
for this_z in zgrid:
    from_fintp_z_mc.append(mass_completion_at_z_from_cat(this_z))
    
from_fintp_z_mc = gaussian_filter1d(from_fintp_z_mc, 1)
def mass_completion_at_z(zred):
    fintp_z_mc = interp1d(zgrid, from_fintp_z_mc, kind='cubic', fill_value="extrapolate")
    return fintp_z_mc(zred)

from_fintp_z_mc_sm = []
for this_z in zgrid:
    from_fintp_z_mc_sm.append(mass_completion_at_z(this_z))
    
plt.plot(zgrid, from_fintp_z_mc_sm, label='Interpolation')
plt.scatter(z_bins_ctr, mc_ctr, s=5, label='Mock catalog', color='k')

plt.xlabel(r'z')
plt.ylabel(r'log $M_{c} \, [M_\odot]$')
plt.legend()
# plt.savefig('figs/mc_z.png', bbox_inches='tight')
plt.show()

# PDF(z)

In [ ]:
def n_at_z_trapz(z0, logmc=None, const_phi=False):
    '''
    const_phi: if True, use L20 massfunctions; if False, use L20+T18 mass functions.
    logmc: if None, then use mass_completion_at_z(z0)
    '''
    if logmc is None:
        logmc_at_z0 = mass_completion_at_z(z0)
    else:
        logmc_at_z0 = logmc * 1
    
    logm_grid = np.linspace(logmc_at_z0, 20, 501)
    phi_50 = PB.mass_func_at_z(z0, logm_grid, const_phi)
    
    n = np.trapz(phi_50, logm_grid)
    return n

zs = np.linspace(0, 20, 501)
n_50s_num = []
for zi in zs:
    n_50s_num.append(n_at_z_trapz(z0=zi, logmc=None, const_phi=const_phi))
dvol = WMAP9.differential_comoving_volume(zs).value
nv = n_50s_num * dvol

if not const_phi:
    # smooth it so we do not have sharp transitions due to Mc(z) being a non-monotonously increasing function.
    nv = gaussian_filter1d(nv, 10)
    
# normalize int_0^20 p(z) = 1
p_int = np.trapz(nv, zs)
pdf_at_z = nv/p_int

invalid = np.where(pdf_at_z<0)
pdf_at_z[invalid] = 0

plt.plot(zs, pdf_at_z)
plt.axvline(0.2, ls=':', c='gray')
plt.axvline(3, ls=':', c='gray')
# plt.yscale('log')
plt.xlabel(r'z')
plt.ylabel(r'pdf')
plt.show()

In [ ]:
if const_phi:
    np.savetxt(prior_data_dir+'pdf_of_z_l20.txt', np.array([zs, pdf_at_z]).T, header='z, probability of observing a galaxy at z')
else:
    np.savetxt(prior_data_dir+'pdf_of_z_l20t18.txt', np.array([zs, pdf_at_z]).T, header='z, probability of observing a galaxy at z')
